## Part 1 : Data Cleaning and Preparation

In [1]:
import numpy as np 
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

#### **Importing data**

In [47]:
df_customer = pd.read_csv('data/profile.csv').drop('Unnamed: 0', axis = 1)
df_offer = pd.read_csv('data/portfolio.csv').drop('Unnamed: 0', axis = 1)
df_transcript = pd.read_csv('data/transcript.csv').drop('Unnamed: 0', axis = 1)

#### **Data Cleaning and Manipulation**
Cleaning and addressing issues each data set, individually.

**df_customer**

In [23]:
df_customer.head(3)

,gender,age,id,became_member_on,income
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN


In [24]:
# Missing data
missing_percent = round(df_customer.isna().mean() * 100, 1)
pd.DataFrame(missing_percent[missing_percent > 0], columns=["% of Missing Values"])

,% of Missing Values
gender,12.8
income,12.8


All of the missing values come from two columns in df_customer: gender and income. The missing values account for 12.8% of the data in each column. All though it's not ideal, the missing values will be removed.

In [25]:
df_customer.dropna(inplace = True)
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14825 entries, 1 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               14825 non-null  int64  
 2   id                14825 non-null  object 
 3   became_member_on  14825 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 694.9+ KB


became_member_on looks like it should be a date, not an integer:

In [26]:
# Data Type Manipulation
df_customer['became_member_on'] = pd.to_datetime(df_customer['became_member_on'], format = '%Y%m%d')

# Knowing that it will probably be helpful during EDA, I'm going to go ahead and add a few day, month, and year columns.
df_customer['year'] = df_customer['became_member_on'].dt.year
df_customer['month_number'] = df_customer['became_member_on'].dt.month
df_customer['day_of_month'] = df_customer['became_member_on'].dt.day
df_customer['month'] = df_customer['became_member_on'].dt.month_name()
df_customer['day_number'] = df_customer['became_member_on'].dt.weekday
df_customer['day'] = df_customer['became_member_on'].dt.day_name()

df_customer.head()

,gender,age,id,became_member_on,income,year,month_number,day_of_month,month,day_number,day
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,2017,7,15,July,5,Saturday
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,2017,5,9,May,1,Tuesday
5,M,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0,2018,4,26,April,3,Thursday
8,M,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0,2018,2,9,February,4,Friday
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,51000.0,2017,11,11,November,5,Saturday


****

**df_offer**

In [27]:
df_offer.head(3)

,reward,channels,difficulty,duration,offer_type,id
0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"['web', 'email', 'mobile', 'social']",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"['web', 'email', 'mobile']",0,4,informational,3f207df678b143eea3cee63160fa8bed


In [28]:
# No missing data
df_offer.isna().sum()

reward        0
channels      0
difficulty    0
duration      0
offer_type    0
id            0
dtype: int64

In [29]:
df_offer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 612.0+ bytes


In [30]:
# Looking at the different offer_types
df_offer['offer_type'].unique()

array(['bogo', 'informational', 'discount'], dtype=object)

In [31]:
# Looking at the different channels
df_offer['channels'].unique()

array(["['email', 'mobile', 'social']",
       "['web', 'email', 'mobile', 'social']",
       "['web', 'email', 'mobile']", "['web', 'email']"], dtype=object)

Since there are three different offer types, we're going to make a categorical column 'offer_code' so that it's easier to refernece each offer type. 

In [32]:
# Creating 'offer_code' for easy referencing
offer_code = {"bogo": 'A', "discount": 'B', "informational": 'C'}
df_offer['offer_code'] = df_offer['offer_type'].map(offer_code)

df_offer.head(3)

,reward,channels,difficulty,duration,offer_type,id,offer_code
0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,A
1,10,"['web', 'email', 'mobile', 'social']",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,A
2,0,"['web', 'email', 'mobile']",0,4,informational,3f207df678b143eea3cee63160fa8bed,C


****

**df_transcript**

In [33]:
df_transcript.head(3)

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0


In [34]:
print(df_transcript['value'])

0         {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1         {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2         {'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3         {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4         {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
                                ...                       
306529                      {'amount': 1.5899999999999999}
306530                                    {'amount': 9.53}
306531                                    {'amount': 3.61}
306532                      {'amount': 3.5300000000000002}
306533                                    {'amount': 4.05}
Name: value, Length: 306534, dtype: object


In [48]:

# Assuming your DataFrame is named df
# Splitting the values based on ":"
#split_values = df_transcript['value'].str.split(':', expand=True)
split_values = df_transcript['value'].str.split(':', n=1, expand=True)

# Renaming the columns
split_values.columns = ['key', 'value']
# Remove curly braces and single quotes from the 'key' and 'value' columns
# Remove curly braces and single quotes from the 'key' and 'value' columns using lambda function
# Remove curly braces and single quotes from the 'key' and 'value' columns using lambda function
split_values['key_id'] = split_values['key'].apply(lambda x: x.replace('{', '').replace('}', '').replace("'", '').strip())
split_values['value_id'] = split_values['value'].apply(lambda x: x.replace('{', '').replace('}', '').replace("'", '').strip())
# Display the cleaned up DataFrame
print(split_values)
# Concatenating the split values DataFrame with the original DataFrame
df_transcript = pd.concat([df_transcript, split_values], axis=1)

# Optionally, dropping the original 'value' column if no longer needed
#df_transcript.drop(columns=['value'], inplace=True)# Optionally, strip whitespace from the key and value columns
# split_values['key'] = split_values['key'].str.strip()
# split_values['value'] = split_values['value'].str.strip()
df_transcript.head()

# # Concatenating the split values DataFrame with the original DataFrame
# df = pd.concat([df_transcript['value'], split_values], axis=1)

# # Optionally, dropping the original 'value' column if no longer needed
# df.drop(columns=['value'], inplace=True)

# # Displaying the resulting DataFrame
# print(df)


                key                                 value    key_id  \
0       {'offer id'   '9b98b8c7a33c4b65b9aebfe6a799e6d9'}  offer id   
1       {'offer id'   '0b1e1539f2cc45b7b9fa7c272da2e1d7'}  offer id   
2       {'offer id'   '2906b810c7d4411798c6938adc9daaa5'}  offer id   
3       {'offer id'   'fafdcd668e3743c1bb461111dcafc2a4'}  offer id   
4       {'offer id'   '4d5c57ea9a6940dd891ad53e9dbe8da0'}  offer id   
...             ...                                   ...       ...   
306529    {'amount'                   1.5899999999999999}    amount   
306530    {'amount'                                 9.53}    amount   
306531    {'amount'                                 3.61}    amount   
306532    {'amount'                   3.5300000000000002}    amount   
306533    {'amount'                                 4.05}    amount   

                                value_id  
0       9b98b8c7a33c4b65b9aebfe6a799e6d9  
1       0b1e1539f2cc45b7b9fa7c272da2e1d7  
2       2906b810c7

,person,event,value,time,key,value,key_id,value_id
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,{'offer id','9b98b8c7a33c4b65b9aebfe6a799e6d9'},offer id,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,{'offer id','0b1e1539f2cc45b7b9fa7c272da2e1d7'},offer id,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,{'offer id','2906b810c7d4411798c6938adc9daaa5'},offer id,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,{'offer id','fafdcd668e3743c1bb461111dcafc2a4'},offer id,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,{'offer id','4d5c57ea9a6940dd891ad53e9dbe8da0'},offer id,4d5c57ea9a6940dd891ad53e9dbe8da0


In [49]:
cleaned_transcript = df_transcript.drop(['value', 'key'], axis = 1)
cleaned_transcript.head()

,person,event,time,key_id,value_id
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,offer id,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,offer id,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,offer received,0,offer id,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,offer id,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,offer id,4d5c57ea9a6940dd891ad53e9dbe8da0


In [17]:
# No missing data
df_transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [18]:
# Change time to hours_since_start
df_transcript = df_transcript.rename(columns = {'time' : 'hours_since_start'})

There's a lot going on in the value column. The column can be separeted by creating a dictionary. This allows us to easily split up the first part ('offer_id') as a dictionary key and second part, the alphanumeric code, as a dictionary value. Essentially, the value column will be split at the colon. 

In [20]:
# Getting value column and turning it into a dictionary
# to get the keys and values
values_col = df_transcript['value']
values_col = values_col.apply(eval)

values_col.sample(3)

79231      {'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'}
130650    {'offer_id': '2906b810c7d4411798c6938adc9daaa5...
15958      {'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
Name: value, dtype: object

In [21]:
# Getting everything before the colon
k = [list(d.keys())[0] for d in values_col]
k[0]

'offer id'

In [22]:
# Getting everything after the colon
v = [list(d.values())[0] for d in values_col]
v[0]

'9b98b8c7a33c4b65b9aebfe6a799e6d9'

In [23]:
# Creating new columns in df_transcript
df_transcript['key_id'] = k
df_transcript['value_id'] = v

In [24]:
# Dropping the original 'value' column since we 
# no longer need it
clean_df_transcript = df_transcript.drop('value', axis = 1)

****

In [25]:
# Save cleaned data for part 2
df_offer.to_csv('Cleaned_Offer.csv')
clean_df_customer.to_csv('Cleaned_Customer.csv')
clean_df_transcript.to_csv('Cleaned_Transcript.csv')